<a href="https://colab.research.google.com/github/2025aa05648/ML_Assignement2/blob/main/2025aa05648_ML_Assignement2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem Statement:** Predict whether a loan application will be approved or rejected based on applicant details such as income, credit history, employment status, education, and loan amount.

**Dataset:** Indian Bank Loan Prediction dataset

**Datasource:** Kaaggle

In [6]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score
from sklearn.metrics import recall_score, f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

import zipfile


In [4]:
#!pip install kaggle

In [ ]:


#os.environ["KAGGLE_USERNAME"] = "saharehemant"
#os.environ["KAGGLE_KEY"] = "KGAT_2b152c052f57b6fbd48275a78b4bc3a8"

#export KAGGLE_API_TOKEN=KGAT_2b152c052f57b6fbd48275a78b4bc3a8
#kaggle competitions list

#!kaggle datasets download -d altruistdelhite04/loan-prediction-problem-dataset

#with zipfile.ZipFile("loan-prediction-problem-dataset.zip", 'r') as zip_ref:
 #   zip_ref.extractall()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
df = pd.read_csv('/content/train.csv')
#print(df.shape)
#df.head()
print(df.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [3]:
#Loading and preprocessing the load data from kaggle




df = pd.read_csv("train.csv")
df.head()

# handling missing value

df.fillna(df.mode().iloc[0], inplace=True)


# Encode categorical variables

le = LabelEncoder()

for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

X = df.drop("Loan_Status", axis=1)
y = df["Loan_Status"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



FileNotFoundError: [Errno 2] No such file or directory: 'loan_data.csv'